# Naive Bayes (the easy way)

In [5]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)
            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []}) 
data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")]);
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

In [7]:
data['spam']=data['class'].apply(lambda x: 1 if x=='spam' else 0)
data

,message,class,spam
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam,1
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam,1
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam,1
emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam,1
emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam,1
...,...,...,...
emails/ham\02496.aae0c81581895acfe65323f344340856,Man killed 'trying to surf' on Tube train \n\n...,ham,0
emails/ham\02497.60497db0a06c2132ec2374b2898084d3,"Hi Gianni,\n\n\n\nA very good resource for thi...",ham,0
emails/ham\02498.09835f512f156da210efb99fcc523e21,Gianni Ponzi wrote:\n\n> I have a prob when tr...,ham,0
emails/ham\02499.b4af165650f138b10f9941f6cc5bce3c,Neale Pickett <neale@woozle.org> writes:\n\n\n...,ham,0


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data.message,data.spam, test_size=0.2)

In [16]:
vectorizer = CountVectorizer()
message_counts = vectorizer.fit_transform(X_train)
print(message_counts)

  (0, 30853)	4
  (0, 41693)	1
  (0, 2146)	1
  (0, 8791)	1
  (0, 1629)	1
  (0, 36282)	1
  (0, 25286)	1
  (0, 45073)	1
  (0, 14377)	3
  (0, 40437)	2
  (0, 28613)	1
  (0, 42619)	1
  (0, 24143)	1
  (0, 29651)	1
  (0, 15067)	3
  (0, 14958)	2
  (0, 38810)	1
  (0, 41129)	15
  (0, 33537)	3
  (0, 45471)	8
  (0, 14371)	5
  (0, 42489)	1
  (0, 12927)	1
  (0, 38597)	1
  (0, 40699)	4
  :	:
  (2399, 35492)	1
  (2399, 23545)	4
  (2399, 23544)	4
  (2399, 29616)	2
  (2399, 9645)	9
  (2399, 21590)	1
  (2399, 37139)	2
  (2399, 34872)	2
  (2399, 8147)	1
  (2399, 13896)	2
  (2399, 36925)	1
  (2399, 17772)	1
  (2399, 26354)	2
  (2399, 21988)	1
  (2399, 44045)	1
  (2399, 20323)	1
  (2399, 24676)	1
  (2399, 11382)	2
  (2399, 31895)	1
  (2399, 36906)	1
  (2399, 10034)	1
  (2399, 8444)	1
  (2399, 29077)	1
  (2399, 29264)	1
  (2399, 34160)	1


In [17]:
classifier = MultinomialNB()
target_class = Y_train

In [18]:
classifier.fit(message_counts, target_class)

MultinomialNB()

In [19]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts1 = vectorizer.transform(examples)
predictions = classifier.predict(example_counts1)
predictions

array([1, 0], dtype=int64)

In [20]:
example_counts = vectorizer.transform(X_test)
predictions = classifier.predict(example_counts)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [21]:
classifier.score(example_counts,Y_test)

0.9716666666666667